In [ ]:
#2.Build a Simple Agentic AI App using LangGraph
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, START, END

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(override=True)
my_api_key = os.getenv("OPENAI_API_KEY")
# print (f'Key is {my_api_key}')


client = OpenAI(api_key=my_api_key)


# --- Shared State ---
class LibraryState(TypedDict):
    question: Optional[str]
    book_recommendation: Optional[str]
    summary_info: Optional[str]
    final_answer: Optional[str]


def ClassifierAgent(state: LibraryState):
    
    print(f"ClassifierAgent Initial state: {state}")

    # Build the LLM messages
    message_to_llm = [
        {"role": "system", "content": '''You are a helpful librarian that recommends 2 top seller based on 
        topic. Decide if the user is asking about book info or summary info. 
        Reply with JSON containing keys: book_recommendation and summary_info.'''},
        {"role": "user", "content": f"Question: {state['question']}"}
    ]
    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=message_to_llm,
        # temperature=0.2,   # keep it deterministic for classification
        # max_tokens=150,
    )
    print (response)
    # Extract the content from the response
    answer = response.choices[0].message.content
    # Ideally, parse as JSON — here assuming model returns a dict-like string
    try:
        import json
        parsed = json.loads(answer)
        print(f"Raw response: {answer}")
        print(f"Parsed response: {parsed}")

        return {
            "book_recommendation": parsed.get("book_recommendation", ""),
            "summary_info": parsed.get("summary_info", "")
        }
    except Exception:
        # fallback if LLM gives plain text
        return {"book_recommendation": answer, "summary_info": ""}

def recommendationAgent(state: LibraryState):
    
    print(f"recommendationAgent Initial state: {state}")
    if not state.get("book_recommendation"):
        return {"book_recommendation": "book recommendation: Not requested"}
    return {"book_recommendation": state["book_recommendation"]}


def summaryAgent(state: LibraryState):
    print(f"summaryAgent Initial state: {state}")
    if not state.get("summary_info"):
        return {"summary_info": "summary info: Not requested"}
    return { "summary_info": state["summary_info"]}


def ResponseAgent(state: LibraryState):
    print(f"ResponseAgent Initial state: {state}")
  
    message_to_llm = [
        {"role": "system", "content": '''You are a response builder for the library application.
          Please combine the book recommendation answer and summary info into a coherent response to the user's question.
        '''},
        {"role": "user", "content": f"recommendation: {state['book_recommendation']}\nsummary Info: {state['summary_info']}\nQuestion: {state['question']}"}
    ]

    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=message_to_llm,
        # temperature=0.2,   # keep it deterministic for classification
        # max_tokens=150,
    )

    print (response)
    # Extract the content from the response
    final_answer = response.choices[0].message.content


    return {"final_answer": final_answer}


# --- Build the Graph ---
builder = StateGraph(LibraryState)
builder.add_node("ClassifierAgent", ClassifierAgent)
builder.add_node("recommendationAgent", recommendationAgent)
builder.add_node("summaryAgent", summaryAgent)
builder.add_node("ResponseAgent", ResponseAgent)

builder.add_edge(START, "ClassifierAgent")
builder.add_edge("ClassifierAgent", "recommendationAgent")
builder.add_edge("ClassifierAgent", "summaryAgent")
builder.add_edge("recommendationAgent", "ResponseAgent")
builder.add_edge("summaryAgent", "ResponseAgent")
builder.add_edge("ResponseAgent", END)

graph = builder.compile()

result = graph.invoke({"question": "Recommend a book about deep learning and summarize it."})
print("\n--- Final Answer ---")
print(result["final_answer"])

ClassifierAgent Initial state: {'question': 'Recommend a book about deep learning and summarize it.'}
ChatCompletion(id='chatcmpl-CVq6UExWg3YYOQAOJpovs3arMyfri', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "book_recommendation": [\n    {\n      "title": "Deep Learning",\n      "authors": ["Ian Goodfellow", "Yoshua Bengio", "Aaron Courville"],\n      "subject": "Foundational text covering theory and practical algorithms in deep learning"\n    },\n    {\n      "title": "Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow",\n      "authors": ["Aurélien Géron"],\n      "subject": "Practical, hands-on guide with code-first approach to building ML and DL models"\n    }\n  ],\n  "summary_info": [\n    {\n      "title": "Deep Learning",\n      "summary": "A comprehensive, mathematically rigorous treatment of deep learning fundamentals. It starts with the mathematical prerequisites, then covers neural networks, backpropag

In [4]:
# # --- Visualize ---
print(graph.get_graph().draw_ascii())
graph.get_graph().draw_png("images/agentic_ai_library.png")
print("Graph saved as agentic_ai_library.png")

                    +-----------+                   
                    | __start__ |                   
                    +-----------+                   
                           *                        
                           *                        
                           *                        
                 +-----------------+                
                 | ClassifierAgent |                
                 +-----------------+                
                  ***            ***                
                **                  **              
              **                      **            
+---------------------+           +--------------+  
| recommendationAgent |           | summaryAgent |  
+---------------------+           +--------------+  
                  ***            ***                
                     **        **                   
                       **    **                     
                  +---------------+           

ImportError: Install pygraphviz to draw graphs: `pip install pygraphviz`.

In [ ]:
#3. Reflection on LangGraph vs AgentKit UI.
#LangGraph involves more code and agent nodes can be connected across diffent LLM's. where as AgentKit has less code and
#can only interact with openAI LLM's.